### IMPORTS

In [1]:
# BASICS
import pandas as pd
from tqdm import tqdm

# GEOPANDAS
import geopandas
from geopandas import GeoSeries
from shapely.geometry import Polygon
import requests

### RECUPERATION REGIONS ET DEPARTEMENTS

In [2]:
# REGION AVEC NOM ET CODE
url_reg = 'https://geo.api.gouv.fr/regions'
r_reg = requests.get(url_reg)
data_reg = r_reg.json()
df_reg = pd.DataFrame(data_reg)

# DEPARTEMENT AVEC NOM, CODE ET CODE REGION
url_dep = 'https://geo.api.gouv.fr/departements'
r_dep = requests.get(url_dep)
data_dep = r_dep.json()
df_dep = pd.DataFrame(data_dep)

### RECUPERATION DES DONNES PAR COMMUNE

In [3]:
# INSTANCE DE GDF
gdf = geopandas.GeoDataFrame()

# COMMUNE POUR CHAQUE DEPARTEMENT VIA GEO API
for dep in tqdm(list(df_dep.code.unique()), desc= "GEO_API"):
    url_com = 'https://geo.api.gouv.fr/departements/'+ dep +'/communes?fields=code,nom,codeEpci,epci,codeDepartement,departement,codeRegion,region,mairie,population,surface,contour&format=geojson&geometry=contour'
    r_com = requests.get(url_com)
    data_com = r_com.json()
    gdf_temp = geopandas.GeoDataFrame.from_features(data_com['features'])
    gdf = pd.concat([gdf_temp, gdf], ignore_index=True)
    
# SUPPRESSION DES DOUBLONS
gdf = gdf.drop_duplicates(subset=['geometry', 'code', 'nom', 'codeEpci', 'codeDepartement', 'codeRegion','population', 'surface'])
print("Doublons supprimés")

# CREATION D'EPCI IMAGINAIRE POUR LES COMMUNES ORPHELINES (CODE INSEE)
gdf['codeEpci'] = gdf['codeEpci'].fillna(value = gdf["code"])

# ---

# ARRONDISSEMENT VIA GEO API
url_arr = 'https://geo.api.gouv.fr/communes?type=arrondissement-municipal&fields=code,nom,codeEpci,epci,codeDepartement,departement,codeRegion,region,mairie,population,surface,contour&format=geojson&geometry=contour'
r_arr = requests.get(url_arr)
data_arr = r_arr.json()
gdf_arr = geopandas.GeoDataFrame.from_features(data_arr['features'])

# CREATION D'EPCI IMAGINAIRE (CODE DEPARTEMENT)
gdf_arr['codeEpci'] = gdf_arr['codeDepartement']

# SUPPRESSION DES VILLES DE PARIS MARSEILLE ET LYON ET REMPLACEMENT AVEC LEUR ARRONDISSEMENT
gdf = gdf[~gdf.nom.isin(["Paris","Marseille","Lyon"])]
gdf = pd.concat([gdf,gdf_arr], ignore_index=True)
print("Arrondissments Paris, Marseille et Lyon remplacés")

GEO_API: 100%|███████████████████████████████████████████████████████████████████████| 101/101 [01:20<00:00,  1.26it/s]


Doublons supprimés
Arrondissments Paris, Marseille et Lyon remplacés


### CARTE EPCI

In [4]:
# Fonctions pour rassembler par départements
def to_epc(gdf,string):
    polygons = gdf.loc[(gdf['codeEpci'] == string)]
    polygons = polygons['geometry'].unary_union
    return polygons

# Création des listes des départements
epcList = list(gdf.codeEpci.unique())

# Instances
shapeList = []
reg_for_epciList = []
dep_for_epciList = []

# Création des formes
for epc in epcList:
    # Forme d'un département
    shape = to_epc(gdf,epc)
    shapeList.append(shape)
    
    # Ajout des régions et département
    dep_for_epciList.append(list(gdf[gdf['codeEpci'] == epc].codeDepartement)[0])
    reg_for_epciList.append(list(gdf[gdf['codeEpci'] == epc].codeRegion)[0])

# Création du GeoDataFrame
data_epc = {'codeRegion':reg_for_epciList, 'codeDepartement': dep_for_epciList, 'codeEpci': epcList, 'geometry': shapeList}
gdf_epc = geopandas.GeoDataFrame(data_epc)

# Ajout des données de populations et surface
gdf_epc_data = gdf.groupby(["codeEpci"]).agg({"population":"sum","surface":"sum"}).reset_index()
gdf_epc = gdf_epc.merge(gdf_epc_data, on='codeEpci')

### CARTE DEPARTEMENT

In [6]:
# Fonctions pour rassembler par départements
def to_dep(gdf,string):
    polygons = gdf.loc[(gdf['codeDepartement'] == string)]
    polygons = polygons['geometry'].unary_union
    return polygons

# Création des listes des départements
depList = list(gdf.codeDepartement.unique())

# Instances
shapeList = []
reg_for_depList = []

# Création des formes
for dep in depList:
    # Forme d'un département
    shape = to_dep(gdf,dep)
    shapeList.append(shape)
    
    # Ajout des régions par département
    reg_for_depList.append(list(gdf[gdf['codeDepartement'] == dep].codeRegion)[0])

# Création du GeoDataFrame
data_dep = {'codeRegion':reg_for_depList, 'codeDepartement': depList, 'geometry': shapeList}
gdf_dep = geopandas.GeoDataFrame(data_dep)

# Ajout des données de populations et surface
gdf_dep_data = gdf.groupby(["codeDepartement"]).agg({"population":"sum","surface":"sum"}).reset_index()
gdf_dep = gdf_dep.merge(gdf_dep_data, on='codeDepartement')

### CARTE REGIONS

In [7]:
def to_reg(gdf,string):
    polygons = gdf.loc[(gdf['codeRegion'] == string)]
    polygons = polygons['geometry'].unary_union
    return polygons

# Création des listes des régions existantes
regList = list(gdf.codeRegion.unique())

# Instances
shapeList = []

# Création des formes
for reg in regList:
    # Forme d'un département
    shape = to_reg(gdf,reg)
    shapeList.append(shape)
    
# Création du GeoDataFrame
data = {'codeRegion': regList, 'geometry': shapeList}
gdf_reg = geopandas.GeoDataFrame(data)

# Ajout des données de populations et surface
gdf_reg_data = gdf.groupby(["codeRegion"]).agg({"population":"sum","surface":"sum"}).reset_index()
gdf_reg = gdf_reg.merge(gdf_reg_data, on='codeRegion')

### RESULTAT

In [8]:
gdf.head(2)

,geometry,code,nom,codeEpci,codeDepartement,codeRegion,population,surface,epci,departement,region
0,"POLYGON ((45.09578 -12.71593, 45.09512 -12.732...",97601,Acoua,200060465,976,06,5192,1254.11,"{'code': '200060465', 'nom': 'CA du Grand Nord...","{'code': '976', 'nom': 'Mayotte'}","{'code': '06', 'nom': 'Mayotte'}"
1,"MULTIPOLYGON (((45.12781 -12.72325, 45.12781 -...",97602,Bandraboua,200060465,976,06,13989,3170.91,"{'code': '200060465', 'nom': 'CA du Grand Nord...","{'code': '976', 'nom': 'Mayotte'}","{'code': '06', 'nom': 'Mayotte'}"


### EXPORTS

In [9]:
print('Exports :')

# EXPORT COMMUNES
gdf.to_file('geo_api_com.geojson', driver='GeoJSON')
print('geo_api_com.geojson exporté')

# EXPORT EPCI
gdf_epc.to_file('geo_api_epc.geojson', driver='GeoJSON')
print('geo_api_epc.geojson exporté')

# EXPORT DEPARTEMENTS
gdf_dep.to_file('geo_api_dep.geojson', driver='GeoJSON')
print('geo_api_dep.geojson exporté')

# EXPORT REGION
gdf_reg.to_file('geo_api_reg.geojson', driver='GeoJSON')
print('geo_api_reg.geojson exporté')

Exports :
geo_api_com.geojson exporté
geo_api_epc.geojson exporté
geo_api_dep.geojson exporté
geo_api_reg.geojson exporté
